In [ ]:
%load_ext autoreload
%autoreload 2
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
import numpy as np
import matplotlib.pyplot as plt
config = {
    "version": 1,
    "controllers": {
        "con1": {
            "type": "opx1",
            "analog_outputs": {
                1: {"offset": +0.0},
                2: {"offset": +0.0},
            },
        }
    },
    "elements": {
        "qe1": {
            "singleInput": {"port": ("con1", 1)},
            "intermediate_frequency": 5e6,
            "operations": {
                "playOp": "constPulse",
            },
        },
        "qe2": {
            "singleInput": {"port": ("con1", 2)},
            "intermediate_frequency": 5e6,
            "operations": {
                "playOp": "constPulse",
            },
        },
    },
    "pulses": {
        "constPulse": {
            "operation": "control",
            "length": 1000,  # in ns
            "waveforms": {"single": "const_wf"},
        },
    },
    "waveforms": {
        "const_wf": {"type": "constant", "sample": 0.2},
    },
}


# Flow control

### For loop

In [ ]:
with program() as prog:
    idx = declare(int)
    a = declare(int)
    with for_(idx,0,idx<10,idx+1):
        assign(a,a+1)
        save(a,"a")
    
    

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")

### While loop

In [ ]:
with program() as prog:
    a = declare(int)
    with while_(a<10):
        assign(a,a+1)
        save(a,"a")
    
    

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")

### Conditional: If and cond

QUA implements an `if - else` which works as you expect: 

In [ ]:
with program() as prog:
    a = declare(int)
    with while_(a<10):
        assign(a,a+1)
        with if_(a==4):
            save(a,"a")
        with else_():
            save(0,"a")
            
    
    

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']



print(f"\nThe program returned the value: a={a}")

**Util.cond** : There is another way to implement simple conditional. The `Util.cond` function is an efficient implementation of the conditional which can speed up operations in some cases:

In [ ]:
with program() as prog:
    a = declare(int,value=0)
    b = declare(int,value=10)
    with while_(a<100):
        assign(a,Util.cond(a == b, a+10000,a+1))
        save(a,"a")
            
    
    

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")

### Conditionally playing pulses

A final conditional usage option is integrated into the play statment itself. By passing a condition to the `play` we play the pulse when a = 0 .. 4 but not for the rest of the loop values. 

In [ ]:
with program() as prog:
    a = declare(int,value=0)
    with for_(a,0,a<10,a+1):
        play('playOp','qe1',condition=(a<5))
        wait(100,'qe1')
        save(a,"a")

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(5000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")



samples = job.get_simulated_samples()
samples.con1.plot()

## Excercises

1. write a program to play two pulses, with delay of 100 cycles between them, repeat 100 times